In [1]:
!pip install gensim

In [2]:
import csv
import pandas as pd
import numpy as np

import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import plotly.express as px

In [115]:
import json

In [222]:
f = open('direct_citation_net.json', encoding='utf-8-sig') # delete descripcion:
network = json.load(f)["network"]

In [223]:
network_items = pd.DataFrame(network["items"])
network_items["DOI"] = network_items["url"].apply(lambda x: x.replace("https://doi.org/", "").lower() if str(x) != "nan" else np.nan)
network_items

,id,label,description,url,x,y,cluster,weights,scores,DOI
0,1,alm k. (2022),<table><tr><td>Authors:</td><td>alm k.; beery ...,https://doi.org/10.1108/ijshe-12-2020-0484,0.8769,-0.1183,1,"{'Links': 4.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 1.0, 'Norm....",10.1108/ijshe-12-2020-0484
1,7,škrinjarić b. (2022),<table><tr><td>Authors:</td><td>škrinjarić b.<...,https://doi.org/10.1057/s41599-022-01047-1,0.6362,0.3087,1,"{'Links': 5.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.1057/s41599-022-01047-1
2,10,leal m.s. (2022),<table><tr><td>Authors:</td><td>leal m.s.; góm...,https://doi.org/10.14201/teri.25394,-0.3645,0.3677,3,"{'Links': 1.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.14201/teri.25394
3,11,birdman j. (2022),<table><tr><td>Authors:</td><td>birdman j.; wi...,https://doi.org/10.1108/ijshe-12-2020-0506,0.8405,-0.0474,1,"{'Links': 5.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 1.0, 'Norm....",10.1108/ijshe-12-2020-0506
4,12,venn r. (2022),<table><tr><td>Authors:</td><td>venn r.; perez...,https://doi.org/10.3390/su14094916,0.4182,-0.0716,1,"{'Links': 6.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.3390/su14094916
...,...,...,...,...,...,...,...,...,...,...
182,880,hyland t. (1997),<table><tr><td>Authors:</td><td>hyland t.</td>...,https://doi.org/10.1111/1467-9752.00070,1.1907,-0.1967,1,"{'Links': 2.0, 'Citations': 55.0, 'Norm. citat...","{'Pub. year': 1997.0, 'Citations': 55.0, 'Norm...",10.1111/1467-9752.00070
183,882,foot s. (1996),<table><tr><td>Authors:</td><td>foot s.; meggi...,https://doi.org/10.1108/00400919610117380,-1.1517,-0.4479,2,"{'Links': 1.0, 'Citations': 4.0, 'Norm. citati...","{'Pub. year': 1996.0, 'Citations': 4.0, 'Norm....",10.1108/00400919610117380
184,885,bridges d. (1996),<table><tr><td>Authors:</td><td>bridges d.</td...,https://doi.org/10.1111/j.1467-9752.1996.tb004...,0.5256,0.3076,1,"{'Links': 1.0, 'Citations': 18.0, 'Norm. citat...","{'Pub. year': 1996.0, 'Citations': 18.0, 'Norm...",10.1111/j.1467-9752.1996.tb00406.x
185,898,hyland t. (1994b),<table><tr><td>Authors:</td><td>hyland t.</td>...,https://doi.org/10.1080/0305569940200208,1.6081,-0.2421,1,"{'Links': 1.0, 'Citations': 6.0, 'Norm. citati...","{'Pub. year': 1994.0, 'Citations': 6.0, 'Norm....",10.1080/0305569940200208


In [224]:
file_name = "scopus.csv"
df_scopus = pd.read_csv(file_name)
df_scopus = df_scopus.reset_index().rename(columns={"index": "id"})
df_scopus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 908 non-null    int64  
 1   Authors            908 non-null    object 
 2   Author(s) ID       907 non-null    object 
 3   Title              908 non-null    object 
 4   Year               908 non-null    int64  
 5   Source title       908 non-null    object 
 6   Volume             871 non-null    object 
 7   Issue              802 non-null    object 
 8   Art. No.           144 non-null    object 
 9   Page start         756 non-null    object 
 10  Page end           756 non-null    object 
 11  Page count         1 non-null      float64
 12  Cited by           724 non-null    float64
 13  DOI                803 non-null    object 
 14  Link               908 non-null    object 
 15  Abstract           908 non-null    object 
 16  Author Keywords    783 non

In [225]:
relevant_columns = {
    "LSA": "Title  Abstract  Author Keywords  Index Keywords".split("  ")
}

In [226]:
df_docs = df_scopus[["id"]+relevant_columns["LSA"]]
df_docs = df_docs.merge(network_items, how="inner", on="id")
df_docs["Abstract"] = df_docs["Abstract"].replace({"[No abstract available]": np.nan})
df_docs = df_docs.fillna("")
df_docs

,id,Title,Abstract,Author Keywords,Index Keywords,label,description,url,x,y,cluster,weights,scores,DOI
0,1,Going beyond the AHA! moment: insight discover...,"In this paper, we develop and apply the concep...",,,alm k. (2022),<table><tr><td>Authors:</td><td>alm k.; beery ...,https://doi.org/10.1108/ijshe-12-2020-0484,0.8769,-0.1183,1,"{'Links': 4.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 1.0, 'Norm....",10.1108/ijshe-12-2020-0484
1,7,Understanding student teachers’ collaborative ...,"Collaborative problem solving, as a key compet...",Computer-mediated communication; Cooperative/c...,Cooperative communication; E-learning; Academi...,škrinjarić b. (2022),<table><tr><td>Authors:</td><td>škrinjarić b.<...,https://doi.org/10.1057/s41599-022-01047-1,0.6362,0.3087,1,"{'Links': 5.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.1057/s41599-022-01047-1
2,10,Developing key competencies in sustainability ...,Purpose: This research aims to investigate the...,Curriculum; Education for sustainable developm...,,leal m.s. (2022),<table><tr><td>Authors:</td><td>leal m.s.; góm...,https://doi.org/10.14201/teri.25394,-0.3645,0.3677,3,"{'Links': 1.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.14201/teri.25394
3,11,Competencies of Sustainability Professionals: ...,This paper presents an empirical study on key ...,higher education; human resource management; k...,higher education; human resource; sustainabili...,birdman j. (2022),<table><tr><td>Authors:</td><td>birdman j.; wi...,https://doi.org/10.1108/ijshe-12-2020-0506,0.8405,-0.0474,1,"{'Links': 5.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 1.0, 'Norm....",10.1108/ijshe-12-2020-0506
4,12,Coaching to prepare students for their school-...,Purpose: The present study proposes coaching a...,Coaching; Employability competences; Higher ed...,,venn r. (2022),<table><tr><td>Authors:</td><td>venn r.; perez...,https://doi.org/10.3390/su14094916,0.4182,-0.0716,1,"{'Links': 6.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.3390/su14094916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,880,The interaction between technical and vocation...,The economies in developed countries have been...,,,hyland t. (1997),<table><tr><td>Authors:</td><td>hyland t.</td>...,https://doi.org/10.1111/1467-9752.00070,1.1907,-0.1967,1,"{'Links': 2.0, 'Citations': 55.0, 'Norm. citat...","{'Pub. year': 1997.0, 'Citations': 55.0, 'Norm...",10.1111/1467-9752.00070
183,882,"Film, adult learning and the prussian peacherino",This article will examine the design and deliv...,,,foot s. (1996),<table><tr><td>Authors:</td><td>foot s.; meggi...,https://doi.org/10.1108/00400919610117380,-1.1517,-0.4479,2,"{'Links': 1.0, 'Citations': 4.0, 'Norm. citati...","{'Pub. year': 1996.0, 'Citations': 4.0, 'Norm....",10.1108/00400919610117380
184,885,"Professionalism, ethics and work-based learning",Recent policy developments within the public s...,Competences; Deprofessionalisation; Profession...,,bridges d. (1996),<table><tr><td>Authors:</td><td>bridges d.</td...,https://doi.org/10.1111/j.1467-9752.1996.tb004...,0.5256,0.3076,1,"{'Links': 1.0, 'Citations': 18.0, 'Norm. citat...","{'Pub. year': 1996.0, 'Citations': 18.0, 'Norm...",10.1111/j.1467-9752.1996.tb00406.x
185,898,Relationship competencies in child welfare ser...,A series of nine focus groups were conducted t...,Child protection; Child welfare; Clients; Comp...,article; child; child welfare; health insuranc...,hyland t. (1994b),<table><tr><td>Authors:</td><td>hyland t.</td>...,https://doi.org/10.1080/0305569940200208,1.6081,-0.2421,1,"{'Links': 1.0, 'Citations': 6.0, 'Norm. citati...","{'Pub. year': 1994.0, 'Citations': 6.0, 'Norm....",10.1080/0305569940200208


In [227]:
df_docs["doc"] = df_docs.apply(lambda x: ". ".join(x.values[1:4+1]), axis=1)
df_docs

,id,Title,Abstract,Author Keywords,Index Keywords,label,description,url,x,y,cluster,weights,scores,DOI,doc
0,1,Going beyond the AHA! moment: insight discover...,"In this paper, we develop and apply the concep...",,,alm k. (2022),<table><tr><td>Authors:</td><td>alm k.; beery ...,https://doi.org/10.1108/ijshe-12-2020-0484,0.8769,-0.1183,1,"{'Links': 4.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 1.0, 'Norm....",10.1108/ijshe-12-2020-0484,Going beyond the AHA! moment: insight discover...
1,7,Understanding student teachers’ collaborative ...,"Collaborative problem solving, as a key compet...",Computer-mediated communication; Cooperative/c...,Cooperative communication; E-learning; Academi...,škrinjarić b. (2022),<table><tr><td>Authors:</td><td>škrinjarić b.<...,https://doi.org/10.1057/s41599-022-01047-1,0.6362,0.3087,1,"{'Links': 5.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.1057/s41599-022-01047-1,Understanding student teachers’ collaborative ...
2,10,Developing key competencies in sustainability ...,Purpose: This research aims to investigate the...,Curriculum; Education for sustainable developm...,,leal m.s. (2022),<table><tr><td>Authors:</td><td>leal m.s.; góm...,https://doi.org/10.14201/teri.25394,-0.3645,0.3677,3,"{'Links': 1.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.14201/teri.25394,Developing key competencies in sustainability ...
3,11,Competencies of Sustainability Professionals: ...,This paper presents an empirical study on key ...,higher education; human resource management; k...,higher education; human resource; sustainabili...,birdman j. (2022),<table><tr><td>Authors:</td><td>birdman j.; wi...,https://doi.org/10.1108/ijshe-12-2020-0506,0.8405,-0.0474,1,"{'Links': 5.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 1.0, 'Norm....",10.1108/ijshe-12-2020-0506,Competencies of Sustainability Professionals: ...
4,12,Coaching to prepare students for their school-...,Purpose: The present study proposes coaching a...,Coaching; Employability competences; Higher ed...,,venn r. (2022),<table><tr><td>Authors:</td><td>venn r.; perez...,https://doi.org/10.3390/su14094916,0.4182,-0.0716,1,"{'Links': 6.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.3390/su14094916,Coaching to prepare students for their school-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,880,The interaction between technical and vocation...,The economies in developed countries have been...,,,hyland t. (1997),<table><tr><td>Authors:</td><td>hyland t.</td>...,https://doi.org/10.1111/1467-9752.00070,1.1907,-0.1967,1,"{'Links': 2.0, 'Citations': 55.0, 'Norm. citat...","{'Pub. year': 1997.0, 'Citations': 55.0, 'Norm...",10.1111/1467-9752.00070,The interaction between technical and vocation...
183,882,"Film, adult learning and the prussian peacherino",This article will examine the design and deliv...,,,foot s. (1996),<table><tr><td>Authors:</td><td>foot s.; meggi...,https://doi.org/10.1108/00400919610117380,-1.1517,-0.4479,2,"{'Links': 1.0, 'Citations': 4.0, 'Norm. citati...","{'Pub. year': 1996.0, 'Citations': 4.0, 'Norm....",10.1108/00400919610117380,"Film, adult learning and the prussian peacheri..."
184,885,"Professionalism, ethics and work-based learning",Recent policy developments within the public s...,Competences; Deprofessionalisation; Profession...,,bridges d. (1996),<table><tr><td>Authors:</td><td>bridges d.</td...,https://doi.org/10.1111/j.1467-9752.1996.tb004...,0.5256,0.3076,1,"{'Links': 1.0, 'Citations': 18.0, 'Norm. citat...","{'Pub. year': 1996.0, 'Citations': 18.0, 'Norm...",10.1111/j.1467-9752.1996.tb00406.x,"Professionalism, ethics and work-based learnin..."
185,898,Relationship competencies in child welfare ser...,A series of nine focus groups were conducted t...,Child protection; Child welfare; Clients; Comp...,article; child; c

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [287]:
import string 

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [284]:
doc = nlp("Teacher, student, group, skill, classroom, learning, assessment, class, environment, outcome")

In [323]:
def preprocess(text):
    tokens = []
    for token in nlp(text):
        val = token.text
        if val not in string.punctuation:
            if not token.is_stop:
                tokens.append(token.lemma_.lower())
    bi_tokens = [f"{tokens[i]}_{tokens[i+1]}" for i in range(len(tokens)-1)]
    return tokens+bi_tokens

In [324]:
df_docs['doc_clean'] = df_docs['doc'].apply(lambda x: preprocess(x))

In [327]:
num_clusters = 3
table_hist = pd.DataFrame()
table_hist["Frequency"] = df_docs["cluster"].value_counts().values
table_hist["Cluster"] = [str(i) for i in range(1, num_clusters+1)]

color_discrete_sequence = ['#d63f4b', "#58bd5b", "#5599c3", "#bcbe4f", "#996fc0", "#5cccd9"][:num_clusters]

fig = px.bar(table_hist, 
             x="Cluster", 
             y="Frequency", 
             color="Cluster", 
             title="Frecuency of clusters (LinLog/mod.)", 
             color_discrete_sequence=color_discrete_sequence)
fig.update_layout(
    font=dict(
        size=18
    )
)
fig.show()

In [328]:
table_hist

,Frequency,Cluster
0,87,1
1,57,2
2,43,3


In [301]:
from gensim import corpora
from gensim import models

In [337]:
cluster_id = 2
df_docs_cluster = df_docs[df_docs["cluster"] == cluster_id]
corpus = df_docs_cluster['doc_clean']
dictionary = corpora.Dictionary(corpus)

bow = [dictionary.doc2bow(text) for text in corpus]

tfidf = models.TfidfModel(bow)
corpus_tfidf = tfidf[bow]

In [303]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

In [330]:
%%time
for i in range(2, 16):
    lsi = LsiModel(corpus_tfidf, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df_docs_cluster['doc_clean'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} topics: {}'.format(i, coherence_score))

Coherence score with 2 topics: 0.4858794562521013
Coherence score with 3 topics: 0.5783951252042933
Coherence score with 4 topics: 0.46818431362331325
Coherence score with 5 topics: 0.43420701194828226


KeyboardInterrupt: 

In [338]:
num_topics = 3
lsi = LsiModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary)

In [339]:
corpus_lsi = lsi[bow]
scores = [[] for _ in range(num_topics)]
for doc in corpus_lsi:
    for k in range(num_topics):
        scores[k].append(round(doc[k][1],2))

df_topic = df_docs_cluster.copy()
for k in range(num_topics):
    df_topic[f'score_topic_{k}'] = scores[k]

df_topic['Topic']= df_topic[[f'score_topic_{k}' for k in range(num_topics)]].apply(lambda x: x.argmax(), axis=1)
df_topic

,id,Title,Abstract,Author Keywords,Index Keywords,label,description,url,x,y,cluster,weights,scores,DOI,doc,doc_clean,score_topic_0,score_topic_1,score_topic_2,Topic
7,17,Practitioner competencies for working with ref...,Research indicates that refugee and asylum see...,Children; competency; practitioner; refugee; y...,asylum seeker; child; cultural competence; edu...,nissilä s.-p. (2022),<table><tr><td>Authors:</td><td>nissilä s.-p.;...,https://doi.org/10.12973/eu-jer.11.2.965,-1.1450,-0.4909,2,"{'Links': 1.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.12973/eu-jer.11.2.965,Practitioner competencies for working with ref...,"[practitioner, competencie, work, refugee, chi...",-5.94,-2.24,1.02,2
12,35,How Key Competencies Progress across School Te...,The recent round of curriculum reform in China...,cognitive development; Geographic competency; ...,,tarekegne c. (2022),<table><tr><td>Authors:</td><td>tarekegne c.; ...,https://doi.org/10.1111/ijtd.12265,-0.6759,-0.2058,2,"{'Links': 5.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",10.1111/ijtd.12265,How Key Competencies Progress across School Te...,"[key, competencies, progress, school, term, st...",-3.81,0.34,2.78,2
14,57,Feasibility and acceptability of virtually coa...,Background: Developing communication skills is...,Communication skills; Resident curriculum; Res...,feasibility study; human; interpersonal commun...,bükki e. (2021),<table><tr><td>Authors:</td><td>bükki e.; fehé...,https://doi.org/10.1186/s40461-020-00108-6,-1.1471,-0.4880,2,"{'Links': 1.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2021.0, 'Citations': 1.0, 'Norm....",10.1186/s40461-020-00108-6,Feasibility and acceptability of virtually coa...,"[feasibility, acceptability, virtually, coach,...",-7.57,-7.54,7.94,2
17,64,Digital competence in higher education: Studen...,University students are expected to have the a...,Digital competence; Higher education; ICT; Stu...,digital mapping; factor analysis; higher educa...,brauer s. (2021),<table><tr><td>Authors:</td><td>brauer s.</td>...,https://doi.org/10.1108/et-07-2020-0216,-0.6495,-0.1055,2,"{'Links': 4.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2021.0, 'Citations': 1.0, 'Norm....",10.1108/et-07-2020-0216,Digital competence in higher education: Studen...,"[digital, competence, high, education, student...",-9.23,1.54,5.00,2
29,128,Botswana teachers: competencies perceived as i...,The implementation of inclusive education cont...,Botswana; inclusive education; teachers’ compe...,,misbah z. (2021),<table><tr><td>Authors:</td><td>misbah z.; gul...,https://doi.org/10.1007/s10984-021-09395-6,-0.9648,-0.3274,2,"{'Links': 2.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2021.0, 'Citations': 1.0, 'Norm....",10.1007/s10984-021-09395-6,Botswana teachers: competencies perceived as i...,"[botswana, teacher, competency, perceive, impo...",-6.84,3.08,-0.91,1
33,135,University graduates: Potential advantage or t...,"The objectives of the study, as presented in t...",Employment and unemployment; Higher education;...,,chen p. (2021),<table><tr><td>Authors:</td><td>chen p.; gonch...,https://doi.org/10.13152/ijrvet.8.4.2,-0.6727,-0.2469,2,"{'Links': 3.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2021.0, 'Citations': 1.0, 'Norm....",10.13152/ijrvet.8.4.2,University graduates: Potential advantage or t...,"[university, graduate, potential, advantage, t...",-5.06,0.73,2.10,2
37,161,The introduction of competence-based education...,"In France, the notion of competence began to b...",Competence; education reform; France; new publ...,comparative study; curriculum; education refor...,diego-mantecón j.-m. (2021),<table><tr><td>Authors:</td><td>diego-mantecón...,https://doi.org/10.3916/c66-2021-03,-1.1900,-0.4417,2,"{'Links': 1.0, 'Citations': 7.0, 'Norm. citati...","{'Pub. year': 2021.0, 'Citations': 7.0, 'Norm....",10.3916/c66-2021-03,The introduction of competence-ba

In [340]:
table_hist_per_topic = pd.DataFrame(df_topic["Topic"].value_counts()).reset_index().rename(columns={"index": "Topic", "Topic": "Frequency"})
table_hist_per_topic["Topic"] = table_hist_per_topic["Topic"].astype(str)
table_hist_per_topic = table_hist_per_topic.sort_values("Topic")

fig = px.bar(table_hist_per_topic, color="Topic", x="Topic", y="Frequency", title=f"Frecuency of topics (Cluster {cluster_id})")
fig.update_layout(
    font=dict(
        size=18
    )
)
fig.show()

In [341]:
data_strengths_per_topic = lsi.print_topics(num_topics)
data_strengths_per_topic

[(0,
  '-0.188*"teacher" + -0.122*"medical" + -0.101*"competence" + -0.100*"competence_base" + -0.100*""" + -0.097*"digital" + -0.093*"competency" + -0.085*"curriculum" + -0.085*"student" + -0.080*"training"'),
 (1,
  '-0.391*"medical" + -0.187*"climate_change" + -0.181*"health" + -0.166*"medical_education" + -0.154*"climate" + 0.145*"teacher" + -0.123*"undergraduate" + -0.098*"education_medical" + -0.092*"resident" + -0.090*"sweden"'),
 (2,
  '0.225*""" + -0.148*"medical" + -0.147*"cbe" + -0.128*"competence_base" + 0.124*"digital" + -0.115*"teacher" + 0.111*"scientific" + 0.102*"skill" + -0.097*"base_education" + -0.089*"education_training"')]

In [342]:
strengths_per_topic = []
for topic_k in range(len(data_strengths_per_topic)):
    id_k, strengths_k = data_strengths_per_topic[topic_k]
    strengths_k = strengths_k.split(" + ")
    for a_w in strengths_k:
        a, w = a_w.split("*")
        a = float(a)
        w=w.replace('"', "")
        o = {"Topic": str(id_k), "Term": w, "Strength": a}
        strengths_per_topic.append(o)

In [343]:
table_strength = pd.DataFrame(strengths_per_topic)
table_strength = table_strength[table_strength["Topic"].isin(relevant_topics)]
table_strength = table_strength.sort_values("Strength", ascending=True)

fig = px.bar(table_strength, color="Topic", y="Term", x="Strength", title=f"Strength of topics terms (Cluster {cluster_id})")
fig.update_layout(
    font=dict(
        size=8
    ),
    height=700,
    width=300
)
fig.show()